In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from keras.models import Model
from keras.layers import LSTM, Activation, Dense, Dropout, Input, Embedding
from keras.optimizers import RMSprop
from keras.preprocessing.text import Tokenizer
from keras.preprocessing import sequence
from keras.utils import pad_sequences
from keras.utils import to_categorical
from keras.callbacks import EarlyStopping


Read dataset and do pre-processing

In [10]:
df = pd.read_csv('/content/drive/MyDrive/IBM/spam.csv',delimiter=',',encoding='latin- 1')
df.head()
df.drop(['Unnamed: 2', 'Unnamed: 3', 'Unnamed: 4'],axis=1,inplace=True)
df.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5572 entries, 0 to 5571
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   v1      5572 non-null   object
 1   v2      5572 non-null   object
dtypes: object(2)
memory usage: 87.2+ KB


Create Model and Add Layers (LSTM, Dense-(Hidden Layers), Output)

In [13]:
X = df.v2 
Y = df.v1
le = LabelEncoder()
Y = le.fit_transform(Y) 
Y = Y.reshape(-1,1)
X_train,X_test,Y_train,Y_test = train_test_split(X,Y,test_size=0.15) 
max_words = 1000
max_len = 150
tok = Tokenizer(num_words=max_words) 
tok.fit_on_texts(X_train)
sequences = tok.texts_to_sequences(X_train) 
sequences_matrix = pad_sequences(sequences,maxlen=max_len)

inputs = Input(name='inputs',shape=[max_len])
layer = Embedding(max_words,50,input_length=max_len)(inputs)
layer = LSTM(64)(layer)
layer = Dense(256,name='FC1')(layer)
layer = Activation('relu')(layer)
layer = Dropout(0.5)(layer)
layer = Dense(1,name='out_layer')(layer)
layer = Activation('sigmoid')(layer)
model = Model(inputs=inputs,outputs=layer)
model.summary()


Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 inputs (InputLayer)         [(None, 150)]             0         
                                                                 
 embedding (Embedding)       (None, 150, 50)           50000     
                                                                 
 lstm (LSTM)                 (None, 64)                29440     
                                                                 
 FC1 (Dense)                 (None, 256)               16640     
                                                                 
 activation (Activation)     (None, 256)               0         
                                                                 
 dropout (Dropout)           (None, 256)               0         
                                                                 
 out_layer (Dense)           (None, 1)                 257   

Compiling the Model

In [14]:
model.compile(loss='binary_crossentropy',optimizer=RMSprop(),metrics=[ 'accuracy'])

Training the Model

In [15]:
model.fit(sequences_matrix,Y_train,batch_size=128,epochs=10, validation_split=0.2)

Epoch 1/10
30/30 [==============================] - 12s 304ms/step - loss: 0.3432 - accuracy: 0.8590 - val_loss: 0.1578 - val_accuracy: 0.9314
Epoch 2/10
30/30 [==============================] - 10s 348ms/step - loss: 0.0953 - accuracy: 0.9744 - val_loss: 0.0392 - val_accuracy: 0.9842
Epoch 3/10
30/30 [==============================] - 9s 306ms/step - loss: 0.0491 - accuracy: 0.9863 - val_loss: 0.0374 - val_accuracy: 0.9916
Epoch 4/10
30/30 [==============================] - 8s 254ms/step - loss: 0.0374 - accuracy: 0.9894 - val_loss: 0.0277 - val_accuracy: 0.9937
Epoch 5/10
30/30 [==============================] - 9s 302ms/step - loss: 0.0317 - accuracy: 0.9913 - val_loss: 0.0234 - val_accuracy: 0.9937
Epoch 6/10
30/30 [==============================] - 8s 271ms/step - loss: 0.0223 - accuracy: 0.9937 - val_loss: 0.0265 - val_accuracy: 0.9916
Epoch 7/10
30/30 [==============================] - 8s 254ms/step - loss: 0.0162 - accuracy: 0.9955 - val_loss: 0.0277 - val_accuracy: 0.9905
Epoc

Saving the Model 

In [16]:
model.save('sms_classifier.h5')

Preprocessing the Test Dataset

In [18]:
test_sequences = tok.texts_to_sequences(X_test) 
test_sequences_matrix = pad_sequences(test_sequences,maxlen=max_len)

Testing the Model

In [19]:
accr = model.evaluate(test_sequences_matrix,Y_test)
print('Test set\n Loss: {:0.3f}\n  Accuracy:{:0.3f}'.format(accr[0],accr[1]))


27/27 [==============================] - 1s 21ms/step - loss: 0.0988 - accuracy: 0.9833
Test set
 Loss: 0.099
  Accuracy:0.983
